### URL to API (https://www.weatherapi.com)

In [3]:
pip install twilio

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import os
from twilio.rest import Client
import time

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

from datetime import datetime

In [5]:
from dotenv import load_dotenv

load_dotenv()

twilio_account_sid = os.getenv('TWILIO_ACCUNT_SID')
twilio_auth_token = os.getenv('TWILIO_AUTH_TOKEN')
phone_number = os.getenv('PHONE_NUMBER')
api_key_wapi = os.getenv('API_KEY_WAPI')

### URL Composition

Get complete information of weather in my city in all hour of day 🌦️

In [6]:
query = 'Bogotá'
api_key = api_key_wapi

url_weather = 'http://api.weatherapi.com/v1/forecast.json?key='+api_key+'&q='+query+'&days=1&aqi=no&alerts=no'

In [7]:
url_response = requests.get(url_weather).json()

In [8]:
url_response.keys()

dict_keys(['location', 'current', 'forecast'])

In [9]:
url_response['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [10]:
len(url_response['forecast']['forecastday'][0]['hour'])  #Hour along to day

24

In [11]:
url_response['forecast']['forecastday'][0]['hour'][1]['time'].split()[0] #Date

'2025-05-20'

In [12]:
int(url_response['forecast']['forecastday'][0]['hour'][1]['time'].split()[1].split(':')[0]) #Hour

1

In [13]:
url_response['forecast']['forecastday'][0]['hour'][22]['condition'] #Condition of weather

{'text': 'Partly Cloudy ',
 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png',
 'code': 1003}

In [14]:
url_response['forecast']['forecastday'][0]['hour'][22]['temp_c'] #Temperature

10.1

In [15]:
url_response['forecast']['forecastday'][0]['hour'][22]['will_it_rain'] #¿Rain?

0

In [16]:
url_response['forecast']['forecastday'][0]['hour'][22]['chance_of_rain'] #Rain probability

0

### DataFrame 

In [17]:
def get_forecast(response, i):
    
    date = url_response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hour = int(url_response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0])
    condition = url_response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    temperature = url_response['forecast']['forecastday'][0]['hour'][i]['temp_c']
    rain = url_response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    prob_rain = url_response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']

    return date, hour, condition, temperature, rain, prob_rain

In [18]:
data_weather = []
len_array = len(url_response['forecast']['forecastday'][0]['hour'])

for i in range(len_array):
    data_weather.append(get_forecast(url_response,i))

In [19]:
data_weather

[('2025-05-20', 0, 'Mist', 10.1, 1, 88),
 ('2025-05-20', 1, 'Mist', 9.7, 1, 71),
 ('2025-05-20', 2, 'Mist', 9.6, 0, 68),
 ('2025-05-20', 3, 'Mist', 9.4, 1, 71),
 ('2025-05-20', 4, 'Mist', 9.3, 0, 0),
 ('2025-05-20', 5, 'Mist', 9.4, 0, 0),
 ('2025-05-20', 6, 'Mist', 9.6, 0, 0),
 ('2025-05-20', 7, 'Partly Cloudy ', 11.4, 0, 0),
 ('2025-05-20', 8, 'Patchy rain nearby', 12.8, 0, 61),
 ('2025-05-20', 9, 'Patchy rain nearby', 13.9, 1, 100),
 ('2025-05-20', 10, 'Light rain shower', 14.9, 1, 100),
 ('2025-05-20', 11, 'Patchy light drizzle', 15.5, 1, 100),
 ('2025-05-20', 12, 'Patchy rain nearby', 16.0, 1, 100),
 ('2025-05-20', 13, 'Patchy rain nearby', 16.0, 1, 100),
 ('2025-05-20', 14, 'Patchy rain nearby', 15.8, 1, 100),
 ('2025-05-20', 15, 'Patchy rain nearby', 15.6, 1, 100),
 ('2025-05-20', 16, 'Patchy rain nearby', 15.2, 1, 100),
 ('2025-05-20', 17, 'Patchy light drizzle', 14.6, 1, 100),
 ('2025-05-20', 18, 'Patchy rain nearby', 11.6, 1, 100),
 ('2025-05-20', 19, 'Mist', 10.4, 1, 72),
 ('

In [20]:
col = ['date', 'hour', 'condition', 'temperature', 'rain', 'prob_rain']
df = pd.DataFrame(data_weather,columns = col)
df

,date,hour,condition,temperature,rain,prob_rain
0,2025-05-20,0,Mist,10.1,1,88
1,2025-05-20,1,Mist,9.7,1,71
2,2025-05-20,2,Mist,9.6,0,68
3,2025-05-20,3,Mist,9.4,1,71
4,2025-05-20,4,Mist,9.3,0,0
5,2025-05-20,5,Mist,9.4,0,0
6,2025-05-20,6,Mist,9.6,0,0
7,2025-05-20,7,Partly Cloudy,11.4,0,0
8,2025-05-20,8,Patchy rain nearby,12.8,0,61
9,2025-05-20,9,Patchy rain nearby,13.9,1,100


In [22]:
df_rain = df[(df['rain']==1) & (df['hour']>6) & (df['hour']<22)] 
df_rain = df_rain[['hour','condition']]
df_rain.set_index('hour', inplace = True)

In [53]:
df_rain = df_rain[-2:]

### Template Message - Send by phone

In [54]:
message_twilio = '\nHola! \n El pronóstico del clima hoy '+df['date'][0]+' en '+query+' es : \n '+str(df_rain)

### Message trhough App Twilio

In [55]:
time.sleep(2)
client = Client(twilio_account_sid, twilio_auth_token)

message = client.messages \
    .create(
        body = message_twilio,
        from_ = phone_number,
        to = '+573105701516'
    )

print('message sent succesful!!' + message.sid)

message sent succesful!!SM0f780b73959c10b0782f8644088c3282
